In [6]:
%run data_read.py

Data path: C:\Users\POP PC\Documents\GitHub\utcc_independent_study\training\..\data\300_data_pop.xlsx
data_size: 318
variable: data train_data test_data


In [7]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer1 = AutoTokenizer.from_pretrained(model_name)

model_name = "pythainlp/thainer-corpus-v2-base-model"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model2 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer2 = AutoTokenizer.from_pretrained(model_name)

model_name = "KoichiYasuoka/roberta-base-thai-spm-upos"
config = AutoConfig.from_pretrained(model_name)
config.num_labels = 1 # regression
model3 = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer3 = AutoTokenizer.from_pretrained(model_name)

loss_fn = torch.nn.L1Loss() # MAE loss

device = torch.device("cpu") # Move your model to the device (CPU)

gradient_accumulation_steps = 4 # Set up gradient accumulation

batch_size = 16  # Reduced batch size
num_epochs = 3

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

airesearch/wangchanberta-base-att-spm-uncased

In [8]:
model = model1
tokenizer = tokenizer1

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 1.4921377152204514
Epoch: 2, Loss: 0.46966568380594254
Epoch: 3, Loss: 0.27748745679855347
Mean Squared Error: 2.5878
Mean Absolute Error: 1.2375
comment: ได้รับสินค้าแล้วว ส่งก่อนกำหนด4วันเลย สินค้าตรงตามปก ราคาดีไม่แพง เคยใช้แล้วชอบ รู้สึกป้องกันลิปเปื้อนแมสได้ดี
actual: 8.0
predicted: 6.225732803344727

comment: ความคุ้มค่า: ดีสินค้าได้รับแล้วส่งมาจากต่างประเทศไม่มีความเสียหายขนส่งมีความทะนุถนอมดีลองทดลองใช้เดี๋ยวขอทดลองใช้ก่อนนะคะว่าเสียงดีไม่ดีแต่ราคาดีค่ะ
actual: 6.0
predicted: 6.223928928375244

comment: จัดส่งไวมากค่ะ ราคาก็ถูกดี เชื่อมต่อง่ายมาก เวลาใช้ค่อนข้างลื่น ไวค่ะ ก็ถนัดดี ไว้จะอุดหนุนใหม่นะคะ
actual: 7.0
predicted: 6.218878269195557

comment: ส่งสินค้ารวดเร็วทันใจ ได้สินค้าตรตามที่สั่ง แพ็คเกจมีความเสียหายเล็กน้อยแตไม่มีผลกับสินค้า ราคาเหมาะสม โดยรวมแล้วถูกใจครับผม โอกาสหน้าค่อยซื้อเพิ่มนะครับ
actual: 8.0
predicted: 6.216344833374023

comment: ได้รับสินค้าเร็วมากภายในไม่กี่วัน เนื่องจากส่งจากในไทยเลย พัสดุกล่อง ยังไม่ได้แกะซองของสินค้า แต่คาดว่าน่าจะใช้

pythainlp/thainer-corpus-v2-base-model

In [9]:
model = model2
tokenizer = tokenizer2

model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 1.2670019716024399
Epoch: 2, Loss: 0.38107942789793015
Epoch: 3, Loss: 0.2404942587018013
Mean Squared Error: 2.5217
Mean Absolute Error: 1.2029
comment: ได้รับสินค้าเร็วมากภายในไม่กี่วัน เนื่องจากส่งจากในไทยเลย พัสดุกล่อง ยังไม่ได้แกะซองของสินค้า แต่คาดว่าน่าจะใช้งานได้ปกติ ไม่น่ามีปัญหาอะไร ประทับใจและจะกลับมาซื้ออีกครับ
actual: 9.0
predicted: 6.314191818237305

comment: สินค้ามีคุณภาพ คุณภาพดี การจัดส่งค่อนข้างรวดเร็วค่ะ ภาพรวมโอเคเลย ได้มีม ถูกใจมากค่ะ สินค้าส่งออก บริหารโดยเ ขนส่งดีค่ะ
actual: 6.0
predicted: 6.313607215881348

comment: สินคเาดีค่ะ ตรงปก ส่งไว แต่หัก1ดาวเพราะสั่งสีฟ้าไปแน่ว่สได้สรขาวมาแทน แต่ว่าไม่มีปัญหาค่ะ สามารถใช้ๆด้ปกติเลย โอเคมาก
actual: 8.0
predicted: 6.313340187072754

comment: สั่งสินค้าไปสีม่วง แต่สีจริงที่ได้รับออกเป็นสีฟ้า ไม่ถูกใจเลย สินค้าใช้ได้ดี แต่คีย์บอร์ดสลับภาษาไม่รู้กดปุ่มไหน ดูตามที่แจ้ง แต่กดเปลี่ยนไม่ได้อ่ะ  งงมาก ส่งของเร็วพอใช้ได้
actual: 6.0
predicted: 6.312061786651611

comment: ได้รับของครบถ้วนค่ะ การจัดส่งไว สินค้าดูดี 

KoichiYasuoka/roberta-base-thai-spm-upos

In [10]:
model = model3
tokenizer = tokenizer3

# Move your model to the device (CPU)
model.to(device)

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Tokenize data and create dataloader
encoded_data = tokenizer.batch_encode_plus(
    train_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=200,
    return_tensors='pt'
)

labels = torch.tensor(train_data['score'].values, dtype=torch.float32)  # Convert labels to float32
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Preprocess the test data
encoded_test_data = tokenizer.batch_encode_plus(
    test_data['comment'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

test_labels = torch.tensor(test_data['score'].values, dtype=torch.float32)
test_dataset = TensorDataset(encoded_test_data['input_ids'], encoded_test_data['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    step = 0

    for batch in train_dataloader:
        step += 1
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps

        loss.backward()

        if step % gradient_accumulation_steps == 0:
            optimizer.step()
            epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / step}')

# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# Evaluate the model's predictions against the actual values
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = logits.squeeze(1).cpu().numpy()

        predictions.extend(preds)
        actuals.extend(labels.cpu().numpy())

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')

result_df = pd.DataFrame({'actual': actuals, 'predicted': predictions})

form = pd.DataFrame(columns=['comment', 'actual', 'predicted'])
count = 0
for i in zip(test_data['comment'], result_df['actual'], result_df['predicted']):
    form.loc[count] = [i[0], i[1], i[2]]
    count += 1

sorted_df = form.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Epoch: 1, Loss: 1.3269223421812057
Epoch: 2, Loss: 0.5231112465262413
Epoch: 3, Loss: 0.41383732110261917
Mean Squared Error: 4.5330
Mean Absolute Error: 1.8593
comment: จากที่ลองใช้มาสักพักดีมากเสียงดีและชัดเสียงไม่แตกเบสหนักฟังเพลงเสียงดีมากมีตำหนินิดหน่อยตรงขอบแต่ไม่เป็นไรแบตทนมากชาร์จ1ครั้งตั้งแต่ได้มาวันแรกจนถึงวันนี้แบตยังเหลืออีก50เปอร์เซ็นต์
actual: 7.0
predicted: 4.947494983673096

comment: สภาพกล่องลำโพงเละเทะมากตั้งเเต่เเกะ ต้องชาร์จลำโพงตลอดเวลาถึงใช้ได้ บลูทูธหลุดบ่อย
actual: 6.0
predicted: 4.898510932922363

comment: ใส่ซิมแล้วไม่มีสัญญาณ ข้อมูลการลงทะเบียนใช้งานไม่ได้ ร้านค้าตอบช้า
actual: 7.0
predicted: 4.8127360343933105

comment: ได้ของมาวันที่13 วันนี้สั่นค้างแล้วก็ดับไปเลย เปิดไม่ติดอีกเลย ลองชาร์จแล้วก็ไม่มีการตอบสนอง
actual: 6.0
predicted: 4.791365146636963

comment: สั่งสินค้าไปสีม่วง แต่สีจริงที่ได้รับออกเป็นสีฟ้า ไม่ถูกใจเลย สินค้าใช้ได้ดี แต่คีย์บอร์ดสลับภาษาไม่รู้กดปุ่มไหน ดูตามที่แจ้ง แต่กดเปลี่ยนไม่ได้อ่ะ  งงมาก ส่งของเร็วพอใช้ได้
actual: 6.0
predicted: 4.7